In [79]:
import os
import sys

proj_root = os.path.abspath(os.getcwd())
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

In [80]:
# Zelle 2: HotelManager importieren und Instanz erstellen

from business_logic.hotel_manager import HotelManager
from data_access.hotel_data_access import HotelDataAccess
import model  # enthält deine Klassen Hotel und Address

# Pfad zur SQLite-Datenbank:
db_path = os.path.join(proj_root, "database", "hotel_reservation_sample.db")

# HotelManager mit dem Datenbankpfad erzeugen
manager = HotelManager(HotelDataAccess(db_path))

1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit
ich dasjenige auswählen kann, welches meinen Wünschen
entspricht. Wünsche sind:


1.1. Ich möchte alle Hotels in einer Stadt durchsuchen,
damit ich das Hotel nach meinem bevorzugten Standort
(Stadt) auswählen kann.


In [ ]:
stadt = "Basel"  # Oder z. B. "Zürich", je nach Datenbankinhalt

hotels = manager.show_hotels_by_city(stadt)

if not hotels:
    print(f"Keine Hotels in {stadt} gefunden.")
else:
    print(f"Gefundene Hotels in {stadt}:\n")
    for h in hotels:
        print(f"- {h.name} ({h.stars} Sterne) – Adresse: {h.address.street}, {h.address.zip_code} {h.address.city}")